# CASA0007 
## Practical 2: Exploratory Data Analysis 2

<div style="float:right"><img width="100" src="https://github.com/jreades/i2p/raw/master/img/casa_logo.jpg" /></div>

## Welcome!

In this practical, we will apply the exploratory data analysis techinques to explore the distribution of datasets. 

- Normal distribution 
- Exponentials 
- Logarithms 

In [1]:
# import relevant packages

import numpy as np
import scipy.stats as sps
import pandas as pd
import matplotlib.pyplot as plt

## Background 

We're going to continue exploring the schools dataset here. 

In [ ]:
# load the data 
df =

## Dataset 1 
### Summary statistics 

We can print the summary statistics of the dataframe using in-built functions from ```pandas```. 

### Data distribution

Let's try and figure out what the distribution of the data is - we start by plotting a histogram. 

In [ ]:
# Plotting the histogram 

It could be helpful to try and change the number of bins - try changing the parameter ```n_bins``` in the below code.  

In [ ]:
# Changing the number of bins in the histogram 

In [ ]:
# Overlay the normal distribution

## Dataset 2

### Summary statistics 

### Data distribution 

In [ ]:
# plot the histogram 

In [ ]:
# overlay the exponential distribution

### Transforming the data - taking the logarithm 

## Conclusions

We have looked at two datasets from the schools data, and explored their distributions. 

Now, can you try applying these techniques to other datasets and describe your findings in the language of data analysis? 

## Extension 

## Credits
### Contributors:
The following individuals have contributed to these teaching materials: [Bea Taylor](https://github.com/Bea-Taylor)

### License
These teaching materials are licensed under a mix of The MIT License and the Creative Commons Attribution-NonCommercial-ShareAlike 4.0 license.

### Acknowledgements
NA

### Dependencies
This notebook depends on the following libraries: pandas, matplotlib, numpy, scipy